In [2]:
import warnings

import numpy as np
import pandas as pd
from darts.models.forecasting.nhits import NHiTSModel
from darts import TimeSeries
import torch
from typing import Callable
from tqdm import tqdm

from helpers import predict, load_agent, quality, clip
from preprocess import preprocess_stats
from rl.sim_enviroment import SimulatedCustomEnv

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.options import DataDriftOptions

/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104warnERKNS_7WarningE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for deta

In [9]:

def optimize_params(data: pd.DataFrame, preprocess: Callable = preprocess_stats, device='cpu') -> pd.DataFrame:
    """
    Run and evaluate agent.

    :param data:        raw observations in pandas DataFrame
    :return:            result saves to the same path as input

    Args:
        preprocess: function to preprocess data
    """
    columns = ['Cell ID', 'LAC', 'HR Usage Rate', 'TCH Blocking Rate, BH', 'Number of Available\nTCH',
               'TCH Traffic (Erl), BH', 'Lower_limit', 'Upper_limit']

    df = preprocess(data, columns)
    obs_array = df.drop(columns=['Cell ID', 'LAC'], errors='ignore')
    obs_array.rename_axis(None, axis=1, inplace=True)
    obs_array.reset_index(drop=True, inplace=True)

    agent = load_agent('sac_last_60_50d_exp-r.pt', 'pt')
    state_predictor = NHiTSModel.load_from_checkpoint("nhits_35lw_2l_1b_3s_35d_no_TB", "state_predictor", best=True, map_location=device)

    # # 'HR Usage Rate', 'TCH Blocking Rate, BH'
    # self.current_state = series[randint(0, len(series))].head(n_past)
    # # 'Number of Available\nTCH', 'TCH Traffic (Erl), BH', 'Param 1',  'Param 2'
    # self.cov = covariates[0].head(n_past)


    lower_limits = []
    upper_limits = []
    qualities = []
    new_states = []

    # 'HR Usage Rate', 'TCH Blocking Rate, BH'
    current_state = obs_array.iloc[:7, :2]
    cov = obs_array.iloc[:7, -4:]

    # print(TimeSeries.from_dataframe(obs_array.iloc[:, :2]))
    # print(len(TimeSeries.from_dataframe(obs_array.iloc[:, :2])))

    # setting env for reward calculation
    environment = SimulatedCustomEnv(
        state_predictor,
        np.array([1,1]),
        TimeSeries.from_dataframe(obs_array.iloc[:, :2]),
        TimeSeries.from_dataframe(obs_array.iloc[:, -4:]),
        7
    )
    obs = environment.reset()
    mom_reward = []

    for i, row in enumerate(obs_array.iloc[7:].values):
        # print('Curr_state=', current_state.shape)

        a1, a2 = predict(row, agent)
        lower = clip(int(row[-2] + a1 * 30))
        upper = clip(int(row[-1] + a2 * 30))

        # compure reward
        new_state, reward, done, info = environment.step(np.array([a1, a2]))
        mom_reward.append(reward)

        # Compute quality
        qualities.append(
            quality(blocking=row[1], ch=row[2], traffic=row[3], param1=row[-2], param2=row[-1], prparam1=lower,
                    prparam2=upper)
        )

        cov.iloc[-1, -2:] = (lower, upper)
        # print(cov)
        # n for number of states to predict
        # current_state.rename_axis(None, axis=1, inplace=True)
        # current_state.reset_index(drop=True, inplace=True)
        pred_state = state_predictor.predict(n=1, series=TimeSeries.from_dataframe(current_state),
                                             past_covariates=TimeSeries.from_dataframe(cov), verbose=False)
        new_states.append(pred_state)

        lower_limits.append(lower)
        upper_limits.append(upper)

        current_state = pd.concat([current_state.iloc[1:], obs_array.iloc[i +7: i+8, :2]], axis=0, join='inner')
        # print(current_state)

        cov = obs_array.iloc[i+1: i +8, -4:]
    # df['Lower_limit_Gen'], df['Upper_limit_Gen'], df['Limit_quality_Gen'] = lower_limits, upper_limits, qualities
    # df["Quality Rate"] = 1 - (2*df['HR Usage Rate']/100 + np.log(df['TCH Blocking Rate, BH'] + 1))/(1 + np.log(101))

    states_df = pd.concat(list(map(lambda x: x.pd_dataframe(), new_states)))
    states_df["Quality Rate"] = 1 - (2*states_df['HR Usage Rate']/100 + np.log(states_df['TCH Blocking Rate, BH'] + 1))/(1 + np.log(101))
    states_df['cum_reward'] = np.cumsum(mom_reward)
    states_df['mom_reward'] = mom_reward

    return states_df


In [4]:
from typing import List


def preprocess_full(data: pd.DataFrame, cols: List[str]=None):
    df = data.copy()
    cols = ['HR Usage Rate', 'TCH Blocking Rate, BH', 'Number of Available\nTCH',
               'TCH Traffic (Erl), BH', 'Lower_limit', 'Upper_limit']
    df.drop(columns='DATA', inplace=True)
    df.rename(columns={'Param 1': cols[-2], 'Param 2': cols[-1]}, inplace=True)
    return df[cols]

In [5]:
df = pd.read_csv('data/dataset_full.csv', index_col=0)

In [6]:
cell_list = list(map(lambda x: x[0], df[['Cell ID']].value_counts().index[:10].tolist()))
curr = df[df['Cell ID'].isin(cell_list)]
reff = df[~df['Cell ID'].isin(cell_list)]

In [7]:
%%time

scores = []

import logging
# logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

    for cell in tqdm(df[['Cell ID']].value_counts().keys()[:2]):
        cell_data = df[df['Cell ID'] == cell]

        data_drift_report = Report(metrics=[
            DataDriftPreset(),
        ])
        data_drift_report.run(reference_data=reff, current_data=cell_data,)
        drift = data_drift_report.as_dict()['metrics'][0]['result']['share_of_drifted_columns']

        states = optimize_params(cell_data, preprocess=preprocess_full)

        scores.append({
            'cell_id': cell,
            'drift_score': drift,
            'quality_avg': states['Quality Rate'].mean(),
            'quality_min': states['Quality Rate'].min(),
            'quality_max': states['Quality Rate'].max(),
            'quality_std': states['Quality Rate'].std(),
            'cum_reward_avg': states['cum_reward'].mean(),
            'cum_reward_max': states['cum_reward'].max(),
            'cum_reward_std': states['cum_reward'].std(),
            'mom_reward_avg': states['mom_reward'].mean(),
            'mom_reward_min': states['mom_reward'].min(),
            'mom_reward_max': states['mom_reward'].max(),
            'mom_reward_std': states['mom_reward'].std(),
        })

scores_df = pd.DataFrame(scores)
scores_df.to_csv('drift_scores_rewards_300.csv')


100%|██████████| 2/2 [01:55<00:00, 57.67s/it]

CPU times: user 3min 4s, sys: 4.99 s, total: 3min 9s
Wall time: 1min 55s


In [7]:
# scores_df = pd.read_csv('drift_scores_rewards.csv', index_col=0)
scores_df

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
0,"(1946,)",0.750,0.874185,0.734750,1.688068,0.056923,-3.575299e+06,-45,3.179021e+06,-17657.961730,-34750,-45,10037.161238
1,"(1945,)",0.750,0.881187,0.707820,1.069511,0.036198,-3.039105e+06,-50,2.671801e+06,-14785.532446,-28105,-50,8152.344845
2,"(1947,)",1.000,0.842564,0.679234,0.999759,0.047799,-1.147139e+06,-10,1.042341e+06,-5578.850000,-9795,-10,3138.280158
3,"(1941,)",0.875,0.852576,0.678216,1.311538,0.069046,-3.435891e+06,-45,3.090324e+06,-17293.177258,-34560,-45,10108.637397
4,"(1943,)",0.875,0.806072,0.668687,1.095661,0.087994,-3.560386e+06,-50,3.162241e+06,-17649.991639,-34895,-50,10007.992846
5,"(1942,)",0.875,0.855051,0.722536,1.801337,0.066887,-3.499136e+06,-60,3.084310e+06,-17133.244147,-32395,-60,9472.583864
6,"(13313,)",0.875,0.777178,0.716126,1.150875,0.034356,-2.864914e+06,-45,2.320279e+06,-12764.756711,-21165,-45,5655.187080
7,"(13312,)",0.875,0.827807,0.683533,1.014011,0.051529,-1.365232e+06,20,1.551018e+06,-8932.390940,-23890,10,7760.800072
8,"(13311,)",0.875,0.766741,0.637207,1.072541,0.050358,-5.250962e+05,15,4.510842e+05,-2500.184564,-4505,10,1293.218758
9,"(22953,)",0.750,0.906204,0.752501,1.126982,0.035694,-3.266940e+06,-45,2.888713e+06,-16287.254237,-30980,-45,9091.123056


In [8]:
scores_df[scores_df.columns[1:]].corr()

,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
drift_score,1.000000,-0.550605,-0.605572,-0.258672,0.269613,0.528747,0.407681,-0.535117,0.541696,0.537498,0.431527,-0.518617
quality_avg,-0.550605,1.000000,0.698381,0.303663,-0.158696,-0.499749,-0.446164,0.543064,-0.546772,-0.565781,-0.449636,0.595814
quality_min,-0.605572,0.698381,1.000000,0.467612,-0.377524,-0.636475,-0.612046,0.626828,-0.625488,-0.561378,-0.623015,0.545800
quality_max,-0.258672,0.303663,0.467612,1.000000,0.323319,-0.563363,-0.528408,0.561030,-0.555300,-0.516847,-0.540661,0.497546
quality_std,0.269613,-0.158696,-0.377524,0.323319,1.000000,-0.303255,-0.185431,0.344766,-0.347661,-0.412466,-0.196013,0.419151
cum_reward_avg,0.528747,-0.499749,-0.636475,-0.563363,-0.303255,1.000000,0.931871,-0.991554,0.988429,0.924603,0.947229,-0.882294
cum_reward_max,0.407681,-0.446164,-0.612046,-0.528408,-0.185431,0.931871,1.000000,-0.886365,0.874997,0.740686,0.997704,-0.678855
cum_reward_std,-0.535117,0.543064,0.626828,0.561030,0.344766,-0.991554,-0.886365,1.000000,-0.999600,-0.965525,-0.907093,0.935646
mom_reward_avg,0.541696,-0.546772,-0.625488,-0.555300,-0.347661,0.988429,0.874997,-0.999600,1.000000,0.971219,0.896998,-0.943279
mom_reward_min,0.537498,-0.565781,-0.561378,-0.516847,-0.412466,0.924603,0.740686,-0.965525,0.971219,1.000000,0.772915,-0.994334


In [23]:
len(df['Cell ID'].unique())

1043

In [11]:
300 / len(df['Cell ID'].unique())

In [10]:
scores_df

In [7]:
import logging

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]

In [13]:
with open('loggers.txt', 'w') as f:
    for item in loggers:
        # write each item on a new line
        f.write("%s\n" % item)

In [20]:
class LessThanFilter(logging.Filter):
    def __init__(self, exclusive_maximum, name=""):
        super(LessThanFilter, self).__init__(name)
        self.max_level = exclusive_maximum

    def filter(self, record):
        #non-zero return means we log this message
        return 1 if record.levelno < self.max_level else 0

logging.getLogger("pytorch_lightning.utilities.rank_zero").addFilter(LessThanFilter(logging.ERROR))

In [23]:
logging.getLogger("pytorch_lightning.utilities.rank_zero").error('asd')
logging.getLogger("pytorch_lightning.utilities.rank_zero").error('asd')
logging.getLogger("pytorch_lightning.utilities.rank_zero").error('asd')
logging.getLogger("pytorch_lightning.utilities.rank_zero").error('asd')
logging.getLogger("pytorch_lightning.utilities.rank_zero").error('asd')

In [27]:
logging.getLogger("pytorch_lightning.utilities.rank_zero").findCaller()

('/home/rid/Soft/anaconda3/envs/sm_bachelor/lib/python3.9/site-packages/IPython/core/interactiveshell.py',
 3448,
 'run_ast_nodes',
 None)

In [9]:
%%time

data_drift_report = Report(metrics=[
    DataDriftPreset(),
])
data_drift_report.run(reference_data=reff, current_data=cell_data,)
drift = data_drift_report.as_dict()['metrics'][0]['result']['share_of_drifted_columns']

CPU times: user 6.43 s, sys: 212 ms, total: 6.65 s
Wall time: 6.8 s


In [10]:
%%time

data_drift_report.run(reference_data=reff, current_data=cell_data,)
drift = data_drift_report.as_dict()['metrics'][0]['result']['share_of_drifted_columns']

CPU times: user 6.22 s, sys: 168 ms, total: 6.39 s
Wall time: 6.49 s


In [11]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    states = optimize_params(cell_data, preprocess=preprocess_full)

CPU times: user 42.4 s, sys: 1.72 s, total: 44.1 s
Wall time: 44.1 s


In [2]:
torch.cuda.is_available()

True

In [10]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    states = optimize_params(cell_data, preprocess=preprocess_full)

CPU times: user 1min 27s, sys: 2.28 s, total: 1min 30s
Wall time: 54.5 s


In [11]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    states = optimize_params(cell_data, preprocess=preprocess_full, device='cuda')

CPU times: user 1min 23s, sys: 1.81 s, total: 1min 25s
Wall time: 48.2 s


# Analysis

In [7]:
data = pd.read_csv('drift_scores_rewards_all.csv', index_col=0)

In [8]:
data

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
0,"(1946,)",0.875,0.874160,0.734750,1.688068,0.056874,-3.587066e+06,-50,3.191874e+06,-17734.276206,-34940,-50,10101.580818
1,"(1945,)",0.875,0.881240,0.712539,1.069511,0.036131,-3.584984e+06,-45,3.210989e+06,-17866.888519,-35685,-45,10343.056816
2,"(1947,)",1.000,0.842714,0.679234,0.999486,0.047754,-1.135826e+06,-10,1.033637e+06,-5546.775000,-9820,-10,3137.497803
3,"(1941,)",0.875,0.852673,0.678700,1.311538,0.068895,-3.447562e+06,-50,3.098796e+06,-17337.090301,-34600,-50,10117.079323
4,"(1943,)",1.000,0.806085,0.669411,1.095661,0.088054,-3.405338e+06,-45,2.981054e+06,-16600.342809,-32240,-45,9092.147583
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,"(12482,)",0.875,0.868466,0.725107,1.062574,0.031370,-3.961857e+05,-45,3.373165e+05,-5342.690476,-10220,-45,2780.442553
1039,"(12483,)",0.875,0.883805,0.840871,0.929947,0.020493,-9.898051e+04,5,9.285494e+04,-2863.457944,-5975,5,1811.103647
1040,"(12481,)",0.875,0.965920,0.886776,1.042712,0.034750,-1.163850e+05,-50,1.039313e+05,-3215.140187,-6395,-50,1861.693343
1041,"(13323,)",0.875,0.981018,0.911628,1.053775,0.032089,-5.948250e+04,-45,5.296255e+04,-2295.000000,-4545,-45,1324.990566


In [9]:
data[data.columns[1:]].corr()

,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
drift_score,1.000000,-0.252896,-0.098583,-0.039432,-0.127550,0.006945,0.068895,-0.021761,0.066041,0.083738,0.066402,-0.093436
quality_avg,-0.252896,1.000000,0.502643,0.333520,0.159484,0.045978,-0.038443,0.009329,-0.147332,-0.238948,-0.042868,0.259073
quality_min,-0.098583,0.502643,1.000000,0.119505,-0.134206,0.070530,-0.045746,-0.026577,-0.088060,-0.172310,-0.047839,0.182302
quality_max,-0.039432,0.333520,0.119505,1.000000,0.468174,-0.152249,-0.073236,0.173904,-0.215231,-0.239303,-0.114423,0.240701
quality_std,-0.127550,0.159484,-0.134206,0.468174,1.000000,-0.085979,-0.064057,0.063141,-0.068361,-0.014295,-0.175647,0.007254
cum_reward_avg,0.006945,0.045978,0.070530,-0.152249,-0.085979,1.000000,0.171343,-0.990619,0.932597,0.820249,0.390343,-0.802359
cum_reward_max,0.068895,-0.038443,-0.045746,-0.073236,-0.064057,0.171343,1.000000,-0.136974,0.164816,0.088091,0.643997,-0.063820
cum_reward_std,-0.021761,0.009329,-0.026577,0.173904,0.063141,-0.990619,-0.136974,1.000000,-0.960482,-0.882851,-0.335615,0.870986
mom_reward_avg,0.066041,-0.147332,-0.088060,-0.215231,-0.068361,0.932597,0.164816,-0.960482,1.000000,0.957747,0.382207,-0.948690
mom_reward_min,0.083738,-0.238948,-0.172310,-0.239303,-0.014295,0.820249,0.088091,-0.882851,0.957747,1.000000,0.236172,-0.995768


In [10]:
data[data.columns[1:]].corr(method='kendall')

,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
drift_score,1.000000,-0.222758,-0.091771,-0.029052,-0.115285,0.072701,0.045894,-0.073499,0.083750,0.090964,0.051647,-0.100316
quality_avg,-0.222758,1.000000,0.369532,0.226132,0.029151,-0.063192,-0.003026,0.080686,-0.144342,-0.197440,-0.006468,0.203410
quality_min,-0.091771,0.369532,1.000000,0.108647,-0.098588,-0.091750,0.017899,0.099998,-0.152579,-0.193956,0.013766,0.190701
quality_max,-0.029052,0.226132,0.108647,1.000000,0.245643,-0.148184,-0.051916,0.153613,-0.176647,-0.184298,-0.048123,0.182747
quality_std,-0.115285,0.029151,-0.098588,0.245643,1.000000,-0.014785,-0.141789,0.008767,-0.001689,0.017910,-0.141768,-0.020737
cum_reward_avg,0.072701,-0.063192,-0.091750,-0.148184,-0.014785,1.000000,0.018777,-0.951919,0.858744,0.737975,0.023881,-0.743357
cum_reward_max,0.045894,-0.003026,0.017899,-0.051916,-0.141789,0.018777,1.000000,-0.003903,-0.004915,-0.033414,0.991089,0.042257
cum_reward_std,-0.073499,0.080686,0.099998,0.153613,0.008767,-0.951919,-0.003903,1.000000,-0.893549,-0.776462,-0.008326,0.787545
mom_reward_avg,0.083750,-0.144342,-0.152579,-0.176647,-0.001689,0.858744,-0.004915,-0.893549,1.000000,0.864664,0.000553,-0.873894
mom_reward_min,0.090964,-0.197440,-0.193956,-0.184298,0.017910,0.737975,-0.033414,-0.776462,0.864664,1.000000,-0.029540,-0.941438


In [11]:
data[data.columns[1:]].corr(method='spearman')

,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
drift_score,1.000000,-0.273376,-0.112515,-0.035694,-0.141639,0.089374,0.048655,-0.090309,0.102738,0.111457,0.054876,-0.122876
quality_avg,-0.273376,1.000000,0.527301,0.333326,0.035460,-0.087325,-0.006718,0.112539,-0.210416,-0.295772,-0.011162,0.305211
quality_min,-0.112515,0.527301,1.000000,0.159504,-0.143087,-0.132457,0.021839,0.143934,-0.220827,-0.283947,0.016713,0.280873
quality_max,-0.035694,0.333326,0.159504,1.000000,0.356822,-0.218206,-0.066568,0.227353,-0.261694,-0.273858,-0.061734,0.271981
quality_std,-0.141639,0.035460,-0.143087,0.356822,1.000000,-0.023141,-0.182319,0.014738,-0.004675,0.026537,-0.182486,-0.030574
cum_reward_avg,0.089374,-0.087325,-0.132457,-0.218206,-0.023141,1.000000,0.026761,-0.995112,0.967544,0.908117,0.033117,-0.908042
cum_reward_max,0.048655,-0.006718,0.021839,-0.066568,-0.182319,0.026761,1.000000,-0.007330,-0.002111,-0.040000,0.993497,0.051562
cum_reward_std,-0.090309,0.112539,0.143934,0.227353,0.014738,-0.995112,-0.007330,1.000000,-0.979258,-0.929143,-0.012881,0.930875
mom_reward_avg,0.102738,-0.210416,-0.220827,-0.261694,-0.004675,0.967544,-0.002111,-0.979258,1.000000,0.972041,0.004592,-0.971656
mom_reward_min,0.111457,-0.295772,-0.283947,-0.273858,0.026537,0.908117,-0.040000,-0.929143,0.972041,1.000000,-0.035342,-0.994058


In [12]:
data.describe()

,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
count,1043.000000,1043.000000,1043.000000,1043.000000,1043.000000,1.043000e+03,1043.000000,1.043000e+03,1043.000000,1043.000000,1043.000000,1043.000000
mean,0.898011,0.860945,0.711063,1.258105,0.057515,-2.911674e+06,-31.299137,2.548311e+06,-14916.982646,-28383.767977,-46.188878,8182.108460
std,0.049690,0.055507,0.035201,0.225152,0.016506,7.938138e+05,228.456211,7.136710e+05,3429.972718,7613.847676,17.025041,2259.314345
min,0.750000,0.718305,0.607482,0.867746,0.015185,-3.587066e+06,-60.000000,5.264149e+04,-17866.888519,-35685.000000,-60.000000,934.315267
25%,0.875000,0.821165,0.688939,1.094380,0.045872,-3.436793e+06,-50.000000,2.317062e+06,-17510.304231,-34815.000000,-50.000000,6326.187334
50%,0.875000,0.854715,0.710457,1.209379,0.055981,-3.302644e+06,-45.000000,2.922087e+06,-16623.737288,-32055.000000,-45.000000,9286.716257
75%,0.875000,0.898882,0.731056,1.373682,0.067125,-2.855031e+06,-45.000000,3.069585e+06,-12960.904458,-23415.000000,-45.000000,10079.644416
max,1.000000,1.037022,0.911628,3.080180,0.178629,-5.892000e+04,5680.000000,3.210989e+06,-1449.232143,-3060.000000,155.000000,10343.056816


In [13]:
data[data.mom_reward_max > 0]

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
7,"(13312,)",1.000,0.827806,0.683533,1.014011,0.051522,-1.488446e+06,5680,1.681490e+06,-9521.929530,-24235,155,7988.393527
8,"(13311,)",1.000,0.766681,0.637207,1.072541,0.050251,-5.236529e+05,15,4.493878e+05,-2489.773490,-4475,10,1283.988759
15,"(22975,)",0.875,0.851060,0.708378,0.965337,0.028175,-5.601129e+05,760,5.161057e+05,-2929.211864,-6100,65,1796.577395
43,"(24233,)",1.000,0.779225,0.680422,1.389400,0.045879,-2.962463e+06,95,2.776694e+06,-15662.177966,-32200,15,9789.926311
102,"(9737,)",0.875,0.836526,0.688055,1.260287,0.039131,-4.723727e+05,310,4.581045e+05,-2602.398305,-5740,35,1746.029977
192,"(42857,)",0.875,0.901327,0.631919,1.170445,0.036514,-1.261592e+06,15,1.248601e+06,-7061.771186,-15505,10,4867.344108
221,"(8916,)",0.875,0.868850,0.690507,0.966925,0.034120,-1.782831e+06,30,1.786634e+06,-10402.563667,-24855,15,7679.013739
248,"(42856,)",0.875,0.925625,0.667989,1.060651,0.031276,-2.937212e+06,5,2.752871e+06,-15626.655348,-32880,5,9852.050424
262,"(41872,)",0.875,0.802087,0.709027,1.287416,0.055670,-3.419002e+06,5,3.045949e+06,-17262.640068,-33845,5,9851.629123
294,"(22971,)",1.000,0.791741,0.703038,1.269978,0.038279,-3.302312e+06,5,2.894209e+06,-16208.149406,-29210,5,8736.894528
